# Reflection


In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the quality of the chosen actions.
This is then used downstream for things like re-planning, search, or evaluation.

#### Prerequisites

We will be using a basic agent with a search tool here.

In [ ]:
%pip install -U --quiet langgraph langchain-openai
%pip install -U --quiet tavily-python

In [2]:
import getpass
import os
from openai import OpenAI

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Reflection"


## Generate

For our example, we will create a "5 paragraph essay" generator. First, create the generator:

In [3]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOpenAI()
generate = prompt | llm

In [4]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

In the timeless classic "The Little Prince" by Antoine de Saint-Exupéry, the whimsical tale of a young prince's journey through the universe offers profound lessons that are as relevant in modern childhood as they were when the book was first published in 1943. Here are some reasons why "The Little Prince" continues to resonate with children today.

Firstly, the Little Prince's exploration of different planets and encounters with various characters highlight the importance of curiosity, imagination, and wonder in a child's life. In today's fast-paced world filled with technology and distractions, the book reminds us of the value of slowing down, observing the world around us, and seeking out the beauty in simple things. Children today can learn from the Little Prince's ability to see things with fresh eyes and appreciate the magic and mystery of life.

Secondly, the book delves into profound themes such as love, friendship, loss, and the complexities of human relationships, which are u

### Reflect

In [5]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [6]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Your essay on why "The Little Prince" is relevant in modern childhood is well-argued and insightful. You have effectively highlighted the timeless themes and lessons found in the book that continue to resonate with children today. 

To enhance your essay, consider the following recommendations:

1. **In-depth analysis**: Dig deeper into specific examples from the book that illustrate its relevance in modern childhood. Provide detailed explanations of how these instances connect with contemporary issues or challenges faced by children today. This will help strengthen your argument and provide a more robust foundation for your claims.

2. **Childhood in the digital age**: Given the prevalence of technology and digital devices in modern childhood, consider exploring how the themes in "The Little Prince" offer a counterbalance to the digital world. Discuss how the book's emphasis on human connection, nature, and emotional depth can serve as a valuable antidote to the distractions and press

### Repeat

And... that's all there is too it! You can repeat in a loop for a fixed number of steps, or use an LLM (or other check) to decide when the finished product is good enough.

In [7]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

In the timeless classic "The Little Prince" by Antoine de Saint-Exupéry, the whimsical tale of a young prince's journey through the universe offers profound lessons that are as relevant in modern childhood as they were when the book was first published in 1943. Here are some reasons why "The Little Prince" continues to resonate with children today.

"The Little Prince" is relevant in modern childhood as it emphasizes the importance of curiosity, imagination, and wonder. In a world dominated by technology and instant gratification, children often find themselves disconnected from the magic of the world around them. The Little Prince's exploration of different planets and encounters with various characters remind children to slow down, observe the beauty in simple things, and approach life with a sense of wonder and curiosity.

Additionally, the themes of love, friendship, loss, and human relationships in the book are universally relevant and particularly poignant in today's society. In 

## Define graph

In [20]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    if len(state) >= 5: # 5 iterations of generation and reflection
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [21]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate an essay on the topicality of The Little Prince and its message in modern life"
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content='"The Little Prince," written by Antoine de Saint-Exupéry, remains a timeless classic that continues to resonate with readers worldwide due to its profound messages and relevance to modern life. The novella, first published in 1943, delves into essential themes such as love, friendship, human nature, and the importance of looking beyond the surface to discover true meaning and beauty in life. In the context of modern society, these messages are perhaps more pertinent than ever, as we grapple with issues of materialism, superficiality, and the loss of connection with what truly matters.\n\nOne of the central messages in "The Little Prince" is the idea of cherishing the intangible and ephemeral aspects of life over material possessions. In a consumer-driven world where value is often equated with material wealth, the little prince\'s simple yet profound observations serve as a poignant reminder of what truly matters. Through his encounters with various char